# NEST Tutorial | Part 3: Connecting networks with synapses

Plot configuration

In [27]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import scienceplots

plt.style.use(["science"])
# mpl.rcParams["font.serif"] = ["Times New Roman"]


### 1. Parameterising synapse models

In [28]:
import nest

nest.ResetKernel()

Synapse model parameters can be viewed, set and copied analogously to neurons:

In [29]:
old_params = nest.GetDefaults("stdp_synapse")
nest.SetDefaults("stdp_synapse", {"tau_plus": 40.0})

new_params = nest.GetDefaults("stdp_synapse")

print(old_params)
print(new_params)


{'alpha': 1.0, 'delay': 1.0, 'has_delay': True, 'lambda': 0.01, 'mu_minus': 1.0, 'mu_plus': 1.0, 'num_connections': 0, 'receptor_type': 0, 'requires_symmetric': False, 'sizeof': 96, 'synapse_model': 'stdp_synapse', 'synapse_modelid': 24, 'tau_plus': 20.0, 'Wmax': 100.0, 'weight': 1.0, 'weight_recorder': ()}
{'alpha': 1.0, 'delay': 1.0, 'has_delay': True, 'lambda': 0.01, 'mu_minus': 1.0, 'mu_plus': 1.0, 'num_connections': 0, 'receptor_type': 0, 'requires_symmetric': False, 'sizeof': 96, 'synapse_model': 'stdp_synapse', 'synapse_modelid': 24, 'tau_plus': 40.0, 'Wmax': 100.0, 'weight': 1.0, 'weight_recorder': ()}


### 2. STDP synapses

Synapse models implementing spike-timing dependent plasticity do not have all of their parameters accessible via GetDefaults() and SetDefaults(), as their dynamics are driven by the postsynaptic and pre-synaptic spike train.

Thus, the time constant of the depressing window of STDP is a parameter os the postsynaptic neuron; set as follows:

In [30]:
nest.Create("iaf_psc_alpha", params={"tau_minus": 30.0})

NodeCollection(metadata=None, model=iaf_psc_alpha, size=1, first=1)

### 3. Connecting with synapse models

Synapse models can be configured during the connection routine. It defaults to _static_synapse_

In [31]:
epop1 = nest.Create("iaf_psc_alpha", n=100)
epop2 = nest.Create("iaf_psc_alpha", n=100)
K = 10

conn_dict = {"rule": "fixed_indegree", "indegree": K}
syn_dict = {"synapse_model": "stdp_synapse", "alpha": 1.0}
nest.Connect(epop1, epop2, conn_dict, syn_dict)

### 4. Distributing synapse parameters

The synapse parameters are specified in the synapse dictionary, which is passed to the Connect()-function. 

Synapse parameters can be set using nest's random module.

In [33]:
alpha_min = 0.1
alpha_max = 2.0
w_min = 0.5
w_max = 5.0

syn_dict = {
    "synapse_model": "stdp_synapse",
    "alpha": nest.random.uniform(min=alpha_min, max=alpha_max),
    "weight": nest.random.uniform(min=w_min, max=w_max),
    "delay": 1.0,
}
nest.Connect(epop1, epop2, "all_to_all", syn_dict)

### 5. Querying the synapses

Since configured rather implicitely, the function

* GetConnections(source=None, target=None, synapse_model=None)

returns a _SynapseCollection_ representing connection identifiers matching given speficiations.

If no specifications are given, all synapses are returned.

**All outgoing connections**

In [34]:
nest.GetConnections(epop1)

**All incoming connections**

In [35]:
nest.GetConnections(target=epop2)

**All synapse models**

In [36]:
nest.GetConnections(synapse_model="stdp_synapse")

Data can be extractd from the SynapseCollection bu calling get(). Here, we retrieve all target ids from stdp_synapse's in epop1 

In [40]:
conns = nest.GetConnections(epop1, synapse_model="stdp_synapse")
targets = conns.get("target")

print(targets)

[175, 127, 143, 144, 128, 122, 110, 158, 174, 129, 126, 111, 177, 125, 179, 145, 176, 142, 124, 146, 130, 201, 109, 177, 173, 180, 123, 122, 147, 157, 123, 141, 131, 178, 176, 121, 148, 159, 112, 181, 108, 140, 132, 120, 179, 149, 172, 133, 119, 107, 118, 200, 150, 156, 171, 180, 139, 113, 175, 182, 117, 117, 134, 151, 118, 116, 121, 183, 138, 106, 181, 124, 135, 115, 152, 136, 182, 114, 170, 153, 199, 137, 174, 114, 105, 155, 160, 184, 113, 112, 154, 183, 137, 111, 155, 136, 126, 154, 125, 138, 169, 104, 110, 178, 198, 139, 173, 185, 184, 129, 109, 156, 185, 168, 115, 108, 157, 172, 103, 186, 135, 140, 107, 158, 153, 187, 186, 134, 141, 120, 106, 105, 159, 167, 197, 142, 126, 102, 161, 116, 187, 104, 133, 160, 171, 188, 143, 103, 194, 188, 166, 102, 161, 132, 144, 196, 152, 201, 162, 189, 117, 170, 189, 162, 200, 199, 163, 131, 145, 190, 190, 146, 165, 198, 164, 151, 127, 197, 191, 195, 164, 130, 147, 165, 196, 169, 191, 118, 195, 192, 165, 166, 129, 119, 148, 194, 149, 150, 128, 160,

In [42]:
conns = nest.GetConnections(epop1, synapse_model="stdp_synapse")
conn_vals = conns.get(["target","weight"])

print(conn_vals)

{'target': [175, 127, 143, 144, 128, 122, 110, 158, 174, 129, 126, 111, 177, 125, 179, 145, 176, 142, 124, 146, 130, 201, 109, 177, 173, 180, 123, 122, 147, 157, 123, 141, 131, 178, 176, 121, 148, 159, 112, 181, 108, 140, 132, 120, 179, 149, 172, 133, 119, 107, 118, 200, 150, 156, 171, 180, 139, 113, 175, 182, 117, 117, 134, 151, 118, 116, 121, 183, 138, 106, 181, 124, 135, 115, 152, 136, 182, 114, 170, 153, 199, 137, 174, 114, 105, 155, 160, 184, 113, 112, 154, 183, 137, 111, 155, 136, 126, 154, 125, 138, 169, 104, 110, 178, 198, 139, 173, 185, 184, 129, 109, 156, 185, 168, 115, 108, 157, 172, 103, 186, 135, 140, 107, 158, 153, 187, 186, 134, 141, 120, 106, 105, 159, 167, 197, 142, 126, 102, 161, 116, 187, 104, 133, 160, 171, 188, 143, 103, 194, 188, 166, 102, 161, 132, 144, 196, 152, 201, 162, 189, 117, 170, 189, 162, 200, 199, 163, 131, 145, 190, 190, 146, 165, 198, 164, 151, 127, 197, 191, 195, 164, 130, 147, 165, 196, 169, 191, 118, 195, 192, 165, 166, 129, 119, 148, 194, 149, 150

### 6. Subsequences and loops

We can configure subsequences of nodes by slicing the relevant population

In [44]:
Nrec = 40
neuronpop = nest.Create("iaf_psc_alpha", n=100)
spikerecorder = nest.Create("spike_recorder")


nest.Connect(neuronpop[:Nrec], spikerecorder, "all_to_all")

Do not configure neurons using for-loops, it is better to work with NodeCollections.